# **Exercicio Duelo de Modelos 4**

### 1. Importando as bases
    As bibliotecas serão importadas conforme a necessidade ao longo do projeto.

In [32]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.info()
test.info()
train.head()
test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [33]:
# Transformando a coluna 'Sex'
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train['Sex'] = le.fit_transform(train['Sex'])
test['Sex'] = le.transform(test['Sex'])

In [34]:
# Substituindo os valores NaN pela mediana em 'Age' e 'Fare'
train['Age'] = train['Age'].fillna(train['Age'].median())
test['Age'] = test['Age'].fillna(test['Age'].median())
test['Fare'] = test['Fare'].fillna(test['Fare'].median())

In [35]:
# Dropando coluna não importante 'Name' e 'Embarked'
train = train.drop(['Name','Embarked'], axis=1)
test = test.drop(['Name','Embarked'], axis=1)

### 2. Aqui faremos um modelo simples de RandomForest (sem alterar nenhum hiperparâmetro) e uparemos a previsão no Kaggle (apenas para comparar futuros resultados).

In [44]:
from sklearn.ensemble import RandomForestClassifier

colunas_para_dropar = ['Survived', 'PassengerId', 'Ticket', 'Cabin']
X_train = train.drop(colunas_para_dropar, axis=1)
y_train = train['Survived']
modelo_1.fit(X_train, y_train)
X_test = test.drop(['PassengerId', 'Ticket', 'Cabin'], axis=1)
predictions = modelo_1.predict(X_test)

In [45]:
# Criando um array para receber o Id do passageiro
submission_df = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': predictions
})

In [38]:
# Salvando o arquivo csv
submission_df.to_csv('submission1.csv', index=False)
# O resultado obtido foi igual ao resultado de algumas tentativas vistas durante o curso. p = 0.74162

![Texto Alternativo](Submission1.png)

### 3. Aqui faremos um outro modelo simples - XGBoost (sem nenhuma alteração de hiperparamêtros inicialmente).


In [46]:
# modelo XGBoost
modelo_2 = XGBClassifier(n_estimators=100, random_state=42)
modelo_2.fit(X_train, y_train)
predictions_2 = modelo_2.predict(X_test)

In [47]:
# Salvando o arquivo csv
submission_df_2 = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': predictions_2
})
submission_df_2.to_csv('submission2.csv', index=False)
# Aqui percebemos que o desempenho foi ligeiramente inferior ao do RandomForest. Acurácia = 0.73205

![Texto Alternativo](Submission2.png)

### 4. Os próximos passos abordados para melhor desempenho de modelos serão:
#### 4.1 - Engenharia de features.
##### 4.1.1 - Viajantes solo ou viajantes em família.
##### 4.1.2 - Agrupando preços do ticket e idade.
#### 4.2 - Ajuste de hiperparâmetros. (serão escolhidos 5 parâmetros para cada modelo)
##### 4.2.1 - Ajustando hiperparâmetros do random forest
##### 4.2.2 - Ajustando hiperparâmetros do XGBoosting

In [55]:
# 4.1.1 - Viajantes solitários ou em família.
# Minha hipótese é de que pessoas que estavam viajando sozinhas tiveram menores chances de sobrevivência - por questões morais.
# reupando e retratando as bases
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
le = LabelEncoder()
train['Sex'] = le.fit_transform(train['Sex'])
test['Sex'] = le.transform(test['Sex'])
train['Age'] = train['Age'].fillna(train['Age'].median())
test['Age'] = test['Age'].fillna(test['Age'].median())
test['Fare'] = test['Fare'].fillna(test['Fare'].median())

In [56]:
# Criando nova coluna 'FamilySize' para ambos os conjuntos
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1

# Criando nova coluna 'IsAlone'
train['IsAlone'] = 0
train.loc[train['FamilySize'] == 1, 'IsAlone'] = 1
test['IsAlone'] = 0
test.loc[test['FamilySize'] == 1, 'IsAlone'] = 1

In [57]:
# 4.1.2 - Agrupando preços do ticket. 
# Minha hipótese é que pessoas que pagaram mais caro tiveram acesso prioritário aos barcos salva-vidas.
# Criando 4 faixas de preço para o ticket
train['Fare_Band'] = pd.qcut(train['Fare'], 4)
test['Fare_Band'] = pd.qcut(test['Fare'], 4)
fare_mapping = {band: i for i, band in enumerate(train['Fare_Band'].unique())}
train['Fare'] = train['Fare_Band'].map(fare_mapping)
test['Fare'] = test['Fare_Band'].map(fare_mapping)

# Removendo a coluna temporária 'Fare_Band'
train.drop('Fare_Band', axis=1, inplace=True)
test.drop('Fare_Band', axis=1, inplace=True)

In [58]:
# 4.1.3 - Agrupando idades.
# Minha hipítese é que pessoas mais velhas foram negligenciadas em primeiro plano, assim como em situações de incêndio, por exemplo.
# Criando 5 faixas etárias
train['Age_Band'] = pd.cut(train['Age'], 5)
test['Age_Band'] = pd.cut(test['Age'], 5)
age_mapping = {band: i for i, band in enumerate(train['Age_Band'].unique())}
train['Age'] = train['Age_Band'].map(age_mapping)
test['Age'] = test['Age_Band'].map(age_mapping)

# Removendo a coluna temporária
train.drop('Age_Band', axis=1, inplace=True)
test.drop('Age_Band', axis=1, inplace=True)

In [63]:
# 4.2.1 - Ajustando hiperparâmetros do RandomForest
from sklearn.model_selection import GridSearchCV

param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}
rf = RandomForestClassifier(random_state=42)
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

print("Melhores hiperparâmetros para o Random Forest:", grid_search_rf.best_params_)

Melhores hiperparâmetros para o Random Forest: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}


In [64]:
# 4.2.2 - Ajustando hiperparâmetros do XGBoosting

param_grid_xgb = {
     'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}
xgb = XGBClassifier(random_state=42)
grid_search_xgb = GridSearchCV(estimator=xgb, param_grid=param_grid_xgb, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_xgb.fit(X_train, y_train)

print("Melhores hiperparâmetros para o XGBoost:", grid_search_xgb.best_params_)

Melhores hiperparâmetros para o XGBoost: {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.9}


### 5. Treinando as bases nessas novas condições

In [65]:
# Criando o RandomForest com os melhores parâmetros e treinando as bases
modelo_1_otimizado = RandomForestClassifier(
    max_depth=10,
    max_features='sqrt',
    min_samples_leaf=1,
    min_samples_split=5,
    n_estimators=100,
    random_state=42
)
modelo_1_otimizado.fit(X_train, y_train)
predictions_rf = modelo_1_otimizado.predict(X_test)

In [66]:
# Salvando o arquivo csv
submission_1 = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': predictions_rf
})
submission_1.to_csv('submission_1_otimizado.csv', index=False)
# Aqui já notamos uma melhora na assertividade do modelo. Acurácia = 0.7703

![Texto Alternativo](Submission_1_otimizado.png)

In [67]:
# Criando o modelo XGBoost com os melhores parâmetros e treinando as bases
modelo_2_otimizado = XGBClassifier(
    colsample_bytree=0.9,
    learning_rate=0.1,
    max_depth=3,
    n_estimators=200,
    subsample=0.9,
    random_state=42
)
modelo_2_otimizado.fit(X_train, y_train)
predictions_xgb = modelo_2_otimizado.predict(X_test)

In [69]:
# Salvando o arquivo csv
submission_2 = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': predictions_xgb
})
submission_2.to_csv('submission_2_otimizado.csv', index=False)
# Aqui não houve variação significativa. Acurácia = 0.74641

![Texto Alternativo](submission_2_otimizado.png)

### Até aqui declaramos como vencedor o modelo RandomForest.
#### Entretanto ainda precisamos
##### 6. Padronização e normalização
##### 7. Balancear as classes
##### 8. Validação cruzada

### 6. Padronizção e normalização

In [77]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### 7. Balancear as classes

In [78]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)

### 8. Validação cruzada

In [81]:
from sklearn.model_selection import cross_val_score

rf_otimizado = RandomForestClassifier(
    max_depth=10,
    max_features='sqrt',
    min_samples_leaf=1,
    min_samples_split=5,
    n_estimators=100,
    random_state=42
)
scores_rf = cross_val_score(rf_otimizado, X_train_balanced, y_train_balanced, cv=5, scoring='accuracy')
print("Pontuações da Validação Cruzada (Random Forest):", scores_rf)

Pontuações da Validação Cruzada (Random Forest): [0.78181818 0.84090909 0.83636364 0.89041096 0.84931507]


In [82]:
from xgboost import XGBClassifier

xgb_otimizado = XGBClassifier(
    colsample_bytree=0.9,
    learning_rate=0.1,
    max_depth=3,
    n_estimators=200,
    subsample=0.9,
    random_state=42
)

scores_xgb = cross_val_score(xgb_otimizado, X_train_balanced, y_train_balanced, cv=5, scoring='accuracy')

print("Pontuações da Validação Cruzada (XGBoost):", scores_xgb)

Pontuações da Validação Cruzada (XGBoost): [0.75909091 0.83181818 0.82727273 0.84474886 0.86757991]


##### A validação cruzada é a prova real de que o randomForest tem um desempenho superior nessa base de dados (confirmando tudo o que foi feito até aqui)

### 9. Treinando os modelos finalizados

In [83]:
modelo_final_rf = RandomForestClassifier(
    max_depth=10,
    max_features='sqrt',
    min_samples_leaf=1,
    min_samples_split=5,
    n_estimators=100,
    random_state=42
)

modelo_final_rf.fit(X_train_balanced, y_train_balanced)
predictions_rf = modelo_final_rf.predict(X_test_scaled)

In [84]:
# 5. Salvando arquivo csv
submission_rf = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': predictions_rf
})
submission_rf.to_csv('submission_rf_final.csv', index=False)
# Acurácia = 0.76794

![Texto Alternativo](rf_final.png)

In [85]:
modelo_final_xgb = XGBClassifier(
    colsample_bytree=0.9,
    learning_rate=0.1,
    max_depth=3,
    n_estimators=200,
    subsample=0.9,
    random_state=42
)
modelo_final_xgb.fit(X_train_balanced, y_train_balanced)
predictions_xgb = modelo_final_xgb.predict(X_test_scaled)

In [86]:
# Salvando arquivo csv
submission_xgb = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': predictions_xgb
})
submission_xgb.to_csv('submission_xgb_final.csv', index=False)
# Acurácia = 0.71770

![Texto Alternativo](xgb_final.png)

### Apenas para testar: VotingClassifier

In [87]:
from sklearn.ensemble import VotingClassifier

voting_model = VotingClassifier(
    estimators=[('rf', rf_otimizado), ('xgb', xgb_otimizado)],
    voting='soft',
    weights=[0.6, 0.4] #
)
voting_model.fit(X_train_balanced, y_train_balanced)
predictions_voting = voting_model.predict(X_test_scaled)

In [89]:
# Salvando o arquivo csv
submission_voting = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': predictions_voting
})
submission_voting.to_csv('submission_voting_final.csv', index=False)
# Acurácia = 0.76315

![Texto Alternativo](voting.png)

##### O modelo de RandomForest se mostrou superior em todos os testes e alterações, a maior Acurácia (.7703) foi obtida no tópico 5.